In [30]:
from IPython.display import Image as IImage
from IPython.display import HTML
import requests
from datetime import datetime, timedelta
import time
from PIL import Image
from __future__ import print_function
import base64
#import cv2
import numpy as np
from __future__ import division

In [4]:
def video(b, mime="video/mp4"):
    encoded = base64.b64encode(b)
    return HTML(data='''<video alt="test" controls>
                    <source src="data:{0};base64,{1}" type="{2}" />
                 </video>'''.format(mime,encoded.decode('ascii'),mime))

In [5]:
scale_base = {
    0  : np.array([0xff, 0xff, 0xff]),
    1  : np.array([0xe4, 0xe5, 0xff]),
    11 : np.array([0x4d, 0x5d, 0xff]),
    21 : np.array([0x00, 0x07, 0x70]),
    25 : np.array([0x00, 0x07, 0x70]),
    26 : np.array([0xfe, 0x16, 0x00]),
    31 : np.array([0xfe, 0x16, 0x00]),
    41 : np.array([0xc0, 0x1c, 0xc4]),
}

scale = np.empty((42, 3), dtype=int)
for i in range(42):
    start_pos = max([j for j in scale_base.keys() if j <= i])
    start = scale_base[start_pos]
    stop_pos = min([j for j in scale_base.keys() if j >= i])
    stop = scale_base[stop_pos]
    
    delta = stop_pos - start_pos
    if delta == 0:
        scale[i] = start
        continue
    ti = i - start_pos
    dist = ti/delta
    color = np.array(np.sqrt((1-dist) * start ** 2 + dist * stop ** 2), dtype=int)
    scale[i] = color

In [6]:
spans = ['<span style="background-color: #{:02x}{:02x}{:02x}; width: 1.5em; height: 3em; display: inline-block;">{}</span>'.format(r,g,b, i) for i,(r, g, b) in enumerate(scale)]
HTML(data="".join(spans))

In [ ]:
from StringIO import StringIO
def make_grayscale(bytes):
    fp = StringIO(bytes)
    img = Image.open(fp).convert("RGBA")
    rgb = np.array(img)
    gray = np.empty(rgb.shape[:2], dtype=np.uint8)
    
    for y in range(gray.shape[0]):
        for x in range(gray.shape[1]):
            pix = rgb[y][x]
            if pix[3] == 0:
                gray[y][x] = 0
            else:
                gray[y][x] = np.linalg.norm(scale - pix[:3], axis=1).argmin() * 5
    
    return Image.fromarray(gray)
    

In [26]:
N = 34
!mkdir -p raw
!rm -f raw/*

now = datetime.utcnow()
delta = timedelta(minutes=(- now.minute % 5))
frame_time = now + delta
for i in range(N):
    t = frame_time + i * timedelta(minutes=5)
    timestr = t.strftime("%Y%m%d%H%M")
    r = requests.get("http://api.buienradar.nl/image/1.0/webmercatorradarnl/png/?t=%s" % timestr)
    if r.ok:
        image = make_grayscale(r.content)
        image.save("gray/%05i.tiff" % i)
        print(".", end="")
    else:
        break

(915, 1058, 4)
.(915, 1058, 4)
.(915, 1058, 4)
.(915, 1058, 4)
.(915, 1058, 4)
.(915, 1058, 4)
.(915, 1058, 4)
.(915, 1058, 4)
.(915, 1058, 4)
.(915, 1058, 4)
.(915, 1058, 4)
.(915, 1058, 4)
.(915, 1058, 4)
.(915, 1058, 4)
.(915, 1058, 4)
.(915, 1058, 4)
.(915, 1058, 4)
.(915, 1058, 4)
.(915, 1058, 4)
.(915, 1058, 4)
.(915, 1058, 4)
.(915, 1058, 4)
.(915, 1058, 4)
.(915, 1058, 4)
.(915, 1058, 4)
.(915, 1058, 4)
.(915, 1058, 4)
.(915, 1058, 4)
.(915, 1058, 4)
.(915, 1058, 4)
.(915, 1058, 4)
.(915, 1058, 4)
.(915, 1058, 4)
.(915, 1058, 4)
.

In [27]:
def trans(s,t):
    A = s.astype(int)
    bornes = list(A.shape[::-1])
    bornes[0] -= 1
    bornes[1] -= 1
    
    T = t.astype(int)
    
    T = np.minimum(T, bornes)
    T = np.maximum(T, [0, 0])

    S = T.reshape(A.size, 2)
    return (
        A.reshape(A.size)[
            S[:,1] * A.shape[1]  + S[:,0] 
        ]
    ).reshape(A.shape)

In [28]:
NFRAMES = 4
!mkdir -p interpolate
!rm -f interpolate/*

destination = np.array(Image.open("gray/%05i.tiff" % 0))

absolute_coorindates = np.empty(destination.shape + (2,), int)
for y in range(absolute_coorindates.shape[0]):
    for x in range(absolute_coorindates.shape[1]):
        absolute_coorindates[y][x][0] = x
        absolute_coorindates[y][x][1] = y
        
j = 0
        
for i in range(3):
    source = destination
    destination = np.array(Image.open("gray/%05i.tiff" % (i+1)))
    relative_flow = cv2.calcOpticalFlowFarneback(source,destination, 0.0, 1, 3, 15, 3, 5, 1, 0)
    flow = absolute_coorindates + relative_flow
    
    Image.fromarray(source).save("interpolate/%05i.tiff" % j)
    j += 1
    
    for ti in np.linspace(0, 1, NFRAMES + 1, endpoint=False)[1:]:
        back = - ti * flow
        forward = (1-ti) * flow
        bf = trans(source,back)
        af = trans(destination, forward)
        frame = (1-ti) * bf + ti * af
        
        Image.fromarray(frame.astype(np.uint8)).save("interpolate/%05i.tiff" % j)
        j += 1

Image.fromarray(destination).save("interpolate/%05i.tiff" % j)

NameError: name 'cv2' is not defined

In [ ]:
destination.dtype

In [ ]:
%%time

NFRAMES = 4

new_frames = []
for t in range(N-1):
    t0 = 0
    t1 = t0 + 1
    flow = flows[t]
    f0 = frames[t][...,1].astype(np.uint16)
    f1 = frames[t+1][...,1].astype(np.uint16)
    new_frames.append(f0)
    
    for ti in np.linspace(0, 1, NFRAMES + 1, endpoint=False)[1:]:

        back = np.array(coords) - ti * flow
        forward = np.array(coords) + (1-ti) * flow
        bf = trans(f0,back)
        af = trans(f1, forward)
        frame = (1-ti) * bf + ti * af
        new_frames.append(frame)
        
new_frames.append(frames[-1][...,1])

In [ ]:
!mkdir -p interpolate
!rm -f interpolate/*
for i,frame in enumerate(new_frames):
    r = Image.fromarray(frame.astype(np.uint8))
    r.save("interpolate/%05i.png" % i)

! ffmpeg -f image2 -r 24 -i interpolate/%05d.png mov/int.webm -y 2> /dev/null

In [31]:
! ffmpeg -f image2 -r 2 -i gray/%05d.png mov/gray.webm -y 2> /dev/null
! ffmpeg -f image2 -r 24 -i interpolate/%05d.png mov/interpolated.webm -y 2> /dev/null

In [ ]:
! ffmpeg -f image2 -r 2 -i gray/%05d.tiff mov/gray.mp4 -y 2> /dev/null

In [ ]:
cap = cv2.VideoCapture("mov/gray.mp4")

In [ ]:
ret, frame2 = cap.read()

In [ ]:
frame2.shape

In [ ]:
frame2.dtype

In [ ]:
gnext = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)

In [ ]:
gnext.dtype

In [ ]:
gnext.shape

In [ ]:
destination.shape

In [ ]:
destination.dtype

In [32]:
scale

array([[255, 255, 255],
       [228, 229, 255],
       [217, 219, 255],
       [206, 209, 255],
       [195, 198, 255],
       [183, 186, 255],
       [170, 174, 255],
       [156, 161, 255],
       [140, 147, 255],
       [123, 131, 255],
       [102, 114, 255],
       [ 77,  93, 255],
       [ 73,  88, 244],
       [ 68,  83, 233],
       [ 64,  77, 221],
       [ 59,  72, 209],
       [ 54,  65, 196],
       [ 48,  59, 183],
       [ 42,  51, 168],
       [ 34,  42, 151],
       [ 24,  30, 133],
       [  0,   7, 112],
       [  0,   7, 112],
       [  0,   7, 112],
       [  0,   7, 112],
       [  0,   7, 112],
       [254,  22,   0],
       [254,  22,   0],
       [254,  22,   0],
       [254,  22,   0],
       [254,  22,   0],
       [254,  22,   0],
       [248,  22,  61],
       [242,  23,  87],
       [237,  23, 107],
       [231,  24, 123],
       [225,  25, 138],
       [218,  25, 151],
       [212,  26, 163],
       [205,  26, 175],
       [199,  27, 185],
       [192,  28